In [14]:
!pip install faker


In [15]:
import pandas as pd
import numpy as np
import random
from faker import Faker

fake = Faker()
np.random.seed(42)
random.seed(42)


N = 100_000

cities = [
    "Lahore", "Karachi", "Islamabad", "Faisalabad",
    "Rawalpindi", "Multan", "Peshawar", "Quetta",
    "lahore ", "KARACHI", "Islambad"  # intentional inconsistencies
]

categories = {
    "Electronics": ["Laptop", "Smartphone", "Tablet", "Headphones"],
    "Clothing": ["T-Shirt", "Jeans", "Jacket", "Shoes"],
    "Home Appliances": ["Microwave", "Refrigerator", "Fan", "Air Conditioner"],
    "Beauty": ["Perfume", "Lipstick", "Face Wash", "Shampoo"]
}

payment_methods = ["Card", "Cash", "Bank Transfer", "Wallet"]

data = []

for i in range(N):

    category = random.choice(list(categories.keys()))
    product = random.choice(categories[category])

    quantity = np.random.randint(1, 6)
    unit_price = round(np.random.uniform(500, 50000), 2)
    discount = np.random.choice([0, 5, 10, 15, 20, 25, 30])

    data.append({
        "order_id": i + 1,
        "order_date": fake.date_between(start_date="-2y", end_date="today"),
        "customer_id": np.random.randint(1000, 5000),
        "city": random.choice(cities),
        "product_name": product,
        "category": category,
        "quantity": quantity,
        "unit_price": unit_price,
        "discount": discount,
        "delivery_days": np.random.randint(1, 11),
        "payment_method": random.choice(payment_methods)
    })

df = pd.DataFrame(data)

# missing values
for col in ["city", "discount", "delivery_days"]:
    df.loc[df.sample(frac=0.01).index, col] = np.nan

# duplicates
df = pd.concat([df, df.sample(500)], ignore_index=True)

df.to_csv("raw_retail_dataset.csv", index=False)

df.head()


,order_id,order_date,customer_id,city,product_name,category,quantity,unit_price,discount,delivery_days,payment_method
0,1,2024-03-08,2095,Rawalpindi,Laptop,Electronics,4,47560.36,10.0,5.0,Cash
1,2,2025-07-12,2482,Karachi,Jeans,Clothing,2,8221.73,10.0,8.0,Card
2,3,2024-06-25,2685,Lahore,Perfume,Beauty,5,30255.19,10.0,5.0,Card
3,4,2024-10-07,3853,lahore,Jeans,Clothing,2,36238.94,25.0,2.0,Card
4,5,2024-09-08,4385,Faisalabad,Shoes,Clothing,4,9578.52,15.0,6.0,Wallet


In [16]:
from google.colab import files
files.download("raw_retail_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
df.shape

(100500, 11)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100500 entries, 0 to 100499
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   order_id        100500 non-null  int64  
 1   order_date      100500 non-null  object 
 2   customer_id     100500 non-null  int64  
 3   city            99494 non-null   object 
 4   product_name    100500 non-null  object 
 5   category        100500 non-null  object 
 6   quantity        100500 non-null  int64  
 7   unit_price      100500 non-null  float64
 8   discount        99495 non-null   float64
 9   delivery_days   99493 non-null   float64
 10  payment_method  100500 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 8.4+ MB


In [20]:
df.isnull().sum()

,0
order_id,0
order_date,0
customer_id,0
city,1006
product_name,0
category,0
quantity,0
unit_price,0
discount,1005
delivery_days,1007


In [21]:
df.duplicated().sum()

np.int64(500)

In [22]:
df.drop_duplicates(inplace=True)

In [23]:
df.duplicated().sum()

np.int64(0)

In [25]:
df['city'] = df['city'].fillna(df['city'].mode()[0])
df['discount'] = df['discount'].median()
df['delivery_days'] = df['delivery_days'].fillna(df['delivery_days'].median())

df['city'] = df['city'].str.strip().str.title()
df['order_date'] = pd.to_datetime(df['order_date'])
print("Cities standardized:", df['city'].unique())
df['total_amount'] = df['quantity'] * df['unit_price']
df['net_amount'] = df['total_amount'] * (1 - df['discount'] / 100)

Cities standardized: ['Rawalpindi' 'Karachi' 'Lahore' 'Faisalabad' 'Islamabad' 'Multan'
 'Islambad' 'Peshawar' 'Quetta']


In [26]:
df.head()

,order_id,order_date,customer_id,city,product_name,category,quantity,unit_price,discount,delivery_days,payment_method,total_amount,net_amount
0,1,2024-03-08,2095,Rawalpindi,Laptop,Electronics,4,47560.36,15.0,5.0,Cash,190241.44,161705.2240
1,2,2025-07-12,2482,Karachi,Jeans,Clothing,2,8221.73,15.0,8.0,Card,16443.46,13976.9410
2,3,2024-06-25,2685,Lahore,Perfume,Beauty,5,30255.19,15.0,5.0,Card,151275.95,128584.5575
3,4,2024-10-07,3853,Lahore,Jeans,Clothing,2,36238.94,15.0,2.0,Card,72477.88,61606.1980
4,5,2024-09-08,4385,Faisalabad,Shoes,Clothing,4,9578.52,15.0,6.0,Wallet,38314.08,32566.9680


In [27]:
output_filename = "cleaned_retail_dataset.csv"
df.to_csv(output_filename, index=False)